In [ ]:
import json
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()

# 임베딩 모델 설정
e_model_id = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=e_model_id)
# e_model_id = "intfloat/multilingual-e5-large"
# embedding_model = HuggingFaceEmbeddings(model=e_model_id)

# Chroma DB 로드
collection_name = "card_info"
persis_directory = "./chroma_db"

vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embedding_model,
    persist_directory=persis_directory,
)

# 카드사 리스트
brands = ["국민", "농협", "롯데", "삼성", "신한", "우리", "하나", "현대", "BC", "IBK기업"]

# 각 카드사 임베딩
for brand in brands:
    with open(f"../data/raw/{brand}.json", "r", encoding="utf-8") as f:
        card_data = json.load(f)

    documents = []
    for card in card_data:
        doc_text = f"{card['name']}는 {card['brand']}에서 발급한 {card['c_brand']} 카드입니다. "
        doc_text += f"연회비는 국내 {card['fee_domestic']}원, 해외겸용 {card['fee_global']}원입니다.\n"
        for b in card['benefits']:
            doc_text += f"- [{b['category']}] {b['short_description']} / {b['detail_description']}\n"
        doc_text += f"카드 신청은 {card['url']}에서 가능합니다.\n"

        documents.append(
            Document(page_content=doc_text, metadata={"name": card['name'], "brand": card['brand'], "c_brand": card['c_brand'], "fee_domestic": card['fee_domestic'], "fee_global": card['fee_global']})
        )
    vector_store.add_documents(documents)
    print(f"{brand}카드 임베딩 추가 완료")

# DB 저장
vector_store.persist()
print("모든 카드 문서가 Chroma DB에 저장되었습니다.")

국민카드 임베딩 추가 완료
농협카드 임베딩 추가 완료
롯데카드 임베딩 추가 완료
삼성카드 임베딩 추가 완료
신한카드 임베딩 추가 완료
우리카드 임베딩 추가 완료
하나카드 임베딩 추가 완료
현대카드 임베딩 추가 완료
BC카드 임베딩 추가 완료
IBK기업카드 임베딩 추가 완료
모든 카드 문서가 Chroma DB에 저장되었습니다.


C:\Users\Playdata\AppData\Local\Temp\ipykernel_7380\2595611233.py:49: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [1]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# e_model_id = "intfloat/multilingual-e5-large"
# embedding_model = HuggingFaceEmbeddings(model=e_model_id)

# Chroma DB 로드
vector_store = Chroma(
    collection_name="card_info",
    embedding_function=embedding_model,
    persist_directory="./chroma_db",
)

# 상위 5개를 검색하는 리트리버 생성
retriever = vector_store.as_retriever(
    search_type="similarity", # 코사인 유사도 검색
    search_kwargs={"k": 5}
)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_25212\2128829386.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [2]:
from langchain.prompts import PromptTemplate

template = """\
당신은 카드 추천 전문가입니다. 
주어진 Context를 바탕으로 질문에 답변해주세요.
Context에 질문에 대한 명확한 정보가 없을 경우 "관련 정보가 없습니다."라고 답변해주세요.
절대 Context에 없는 내용을 추측하거나 일반 상식을 이용해 답을 만들어서 대답하지 않습니다.

Context:{context}

질문: {question}

추천 카드 목록을 아래 형식으로 출력해주세요.
- 카드명: [카드명]
- 카드사: [카드사]
- 연회비: [연회비]
- 혜택: [혜택 요약]
- 카드 자세히 보기: [URL]
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from huggingface_hub import login
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import pipeline

from dotenv import load_dotenv
import os
load_dotenv()
# hf_token = os.getenv("HUGGINGFACE_API_KEY")
# login(hf_token)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
# model_id = "google/gemma-3-1b-it"
# generator = pipeline(
#     task="text-generation",
#     model=model_id,
#     tokenizer=model_id
# )
# llm = HuggingFacePipeline(pipeline=generator)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [4]:
question = "간편결제 혜택과 디지털 구독 서비스가 모두 있는 카드 중 베스트 1개 추천해줘"
response = rag_chain.invoke(question)

print(response)

- 카드명: 삼성 iD GLOBAL 카드  
- 카드사: 삼성카드  
- 연회비: 국내 20,000원, 해외겸용 20,000원  
- 혜택:  
  - 간편결제: 삼성페이, 네이버페이, 카카오페이 온라인 간편결제 1% 할인 (일상 필수 영역)  
  - 디지털 구독: 인앱 결제/디지털콘텐츠/멤버십 50% 결제일 할인 (넷플릭스, 디즈니+, 유튜브 프리미엄, 쿠팡 로켓와우 등)  
  - 해외 수수료 할인 및 해외 2% 할인, 삼성페이 해외 오프라인 5% 할인  
  - 공항 라운지 무료 이용 (월 1회, 연 2회)  
- 카드 자세히 보기: https://www.card-gorilla.com/card/detail/2676


In [6]:
question = "반려동물 보험 혜택이 있는 카드는 뭐야?"
response = rag_chain.invoke(question)

print(response)

관련 정보가 없습니다.


In [7]:
question = input("질문을 입력하세요: ")
response = rag_chain.invoke(question)

print(response)

- 카드명: K-패스 신한카드  
- 카드사: 신한카드  
- 연회비: 국내 7,000원 / 해외겸용 10,000원  
- 혜택: 해외겸용 Mastercard 카드로 해외 결제 시 간편결제 5% 결제일 할인(단, 해외 간편결제 및 교통카드 이용금액 제외), 대중교통 10% 할인, 생활서비스(배달앱, 편의점, 커피전문점, 올리브영, 병원/약국, OTT, 이동통신요금 자동이체) 5% 결제일 할인  
- 카드 자세히 보기: https://www.card-gorilla.com/card/detail/2690

- 카드명: zgm.휴가중카드  
- 카드사: NH농협카드  
- 연회비: 국내 25,000원 / 해외겸용 25,000원  
- 혜택: 해외 이용액 2~5% NH포인트 적립(전월실적에 따라 차등적립), 국내 이용액 0.5% 적립, 항공권·면세점 0.5% 추가적립, 호텔스닷컴 20% 즉시할인, 전 세계 공항 라운지 월 1회 무료 이용(연 2회), 여행자보험 무료 가입  
- 카드 자세히 보기: https://www.card-gorilla.com/card/detail/2464

- 카드명: BC 바로 On&Off 카드  
- 카드사: BC 바로카드  
- 연회비: 국내 5,000원 / 해외겸용 5,000원  
- 혜택: 해외 온/오프라인 가맹점 10% 결제일 할인, 국내 온라인 및 간편결제 10% 할인, 버스/지하철/택시 10% 할인, 음식점/커피 10% 할인, 2~3개월 무이자할부 서비스 제공  
- 카드 자세히 보기: https://www.card-gorilla.com/card/detail/2591


In [8]:
# RAGAS 평가

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# 질문 리스트
questions = [
    "대중교통이랑 커피 할인 되는 카드 추천해줘",
    "반려동물 보험 혜택 있는 카드가 있어?",
    "해외여행 시 유용한 카드 추천해줘",
    "연회비가 10만원 이하인 카드 중에서 추천해줘",
    "디지털 구독 서비스 혜택이 있는 카드가 뭐야?",
]

# 실제 답변과 문맥 수집
answers = []
contexts = []
for q in questions:
    result = rag_chain.invoke(q)
    answers.append(result)

    docs = retriever.get_relevant_documents(q)
    contexts.append([d.page_content for d in docs])

# 평가용 데이터셋 구성
dataset = Dataset.from_dict({
    "question": questions,
    "answer": answers,
    "contexts": contexts,
})

metrics = [faithfulness, answer_relevancy]
results = evaluate(dataset=dataset, metrics=metrics)

print(results.to_pandas())

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

                  user_input  \
0    대중교통이랑 커피 할인 되는 카드 추천해줘   
1      반려동물 보험 혜택 있는 카드가 있어?   
2         해외여행 시 유용한 카드 추천해줘   
3  연회비가 10만원 이하인 카드 중에서 추천해줘   
4  디지털 구독 서비스 혜택이 있는 카드가 뭐야?   

                                  retrieved_contexts  \
0  [K-패스 (신용)는 IBK기업은행에서 발급한 Mastercard 카드입니다. 연회...   
1  [K-패스 (신용)는 IBK기업은행에서 발급한 Mastercard 카드입니다. 연회...   
2  [BC 바로 On&Off 카드는 BC 바로카드에서 발급한 VISA 카드입니다. 연회...   
3  [카드의정석 TEN는 우리카드에서 발급한 Mastercard 카드입니다. 연회비는 ...   
4  [현대카드 M는 현대카드에서 발급한 VISA 카드입니다. 연회비는 국내 30,000...   

                                            response  faithfulness  \
0  - 카드명: K-패스 (신용)  \n- 카드사: IBK기업은행  \n- 연회비: 국...      0.250000   
1                                       관련 정보가 없습니다.      1.000000   
2  - 카드명: 카드의정석 EVERY MILE SKYPASS  \n- 카드사: 우리카드...      0.857143   
3  - 카드명: LOCA LIKIT Eat  \n- 카드사: 롯데카드  \n- 연회비:...      0.952381   
4  - 카드명: LOCA 365 카드  \n- 카드사: 롯데카드  \n- 연회비: 국내...      0.937500   

   answer_relevancy  
0          0.798754  
1    

In [9]:
results.to_pandas()

,user_input,retrieved_contexts,response,faithfulness,answer_relevancy
0,대중교통이랑 커피 할인 되는 카드 추천해줘,[K-패스 (신용)는 IBK기업은행에서 발급한 Mastercard 카드입니다. 연회...,- 카드명: K-패스 (신용) \n- 카드사: IBK기업은행 \n- 연회비: 국...,0.250000,0.798754
1,반려동물 보험 혜택 있는 카드가 있어?,[K-패스 (신용)는 IBK기업은행에서 발급한 Mastercard 카드입니다. 연회...,관련 정보가 없습니다.,1.000000,0.000000
2,해외여행 시 유용한 카드 추천해줘,[BC 바로 On&Off 카드는 BC 바로카드에서 발급한 VISA 카드입니다. 연회...,- 카드명: 카드의정석 EVERY MILE SKYPASS \n- 카드사: 우리카드...,0.857143,0.798802
3,연회비가 10만원 이하인 카드 중에서 추천해줘,[카드의정석 TEN는 우리카드에서 발급한 Mastercard 카드입니다. 연회비는 ...,- 카드명: LOCA LIKIT Eat \n- 카드사: 롯데카드 \n- 연회비:...,0.952381,0.797970
4,디지털 구독 서비스 혜택이 있는 카드가 뭐야?,"[현대카드 M는 현대카드에서 발급한 VISA 카드입니다. 연회비는 국내 30,000...",- 카드명: LOCA 365 카드 \n- 카드사: 롯데카드 \n- 연회비: 국내...,0.937500,0.846725
